In [1]:
# This notebook is from Prophesee toolbox ;)
# always move notebook to .. 
import numpy as np
from prophesee.src.io.psee_loader import PSEELoader
from prophesee.src.io.box_loading import reformat_boxes
from settings.data import settings
import os.path as op
import torch

In [2]:
video = PSEELoader(op.join(settings['automotive_path'],"train_a","17-03-30_12-53-58_1098500000_1158500000_td.dat"))
print(video)  # show some metadata
video.event_count()  # number of events in the file
video.total_time()  # duration of the file in mus

PSEELoader:
-----------
Event Type: Event2D
Event Size: 8 bytes
Event Count: 63200927
Duration: 59.999998999999995 s 
-----------



59999999

# Voxels
Convert events to voxels.

References:

https://openaccess.thecvf.com/content_CVPR_2019/papers/Zhu_Unsupervised_Event-Based_Learning_of_Optical_Flow_Depth_and_Egomotion_CVPR_2019_paper.pdf

https://github.com/alexzzhu/EventGAN/blob/master/EventGAN/utils/event_utils.py


In [3]:
def us_to_s(us):
    return us/1000000

def us_to_ms(us):
    return us/1000

def s_to_us(us):
    return us*1000000

def ms_to_us(us):
    return us*1000

def calc_floor_ceil_delta(x): 
    # github.com/alexzzhu/EventGAN
    x_fl = torch.floor(x + 1e-8)
    x_ce = torch.ceil(x - 1e-8)
    x_ce_fake = torch.floor(x) + 1

    dx_ce = x - x_fl
    dx_fl = x_ce_fake - x
    return [x_fl.long(), dx_fl], [x_ce.long(), dx_ce]

def create_update(x, y, t, dt, p, vol_size):
    # github.com/alexzzhu/EventGAN
    assert (x>=0).byte().all() and (x<vol_size[2]).byte().all()
    assert (y>=0).byte().all() and (y<vol_size[1]).byte().all()
    assert (t>=0).byte().all() and (t<vol_size[0] // 2).byte().all()

    vol_mul = torch.where(p < 0,
                          torch.ones(p.shape, dtype=torch.long) * vol_size[0] // 2,
                          torch.zeros(p.shape, dtype=torch.long))

    inds = (vol_size[1]*vol_size[2]) * (t + vol_mul)\
         + (vol_size[2])*y\
         + x

    vals = dt

    return inds, vals

def events_to_voxel(events, bins, height, width, device=torch.device('cuda:0')):
    # github.com/alexzzhu/EventGAN
    vol_size = [2*bins, height, width]
    npts = events.shape[0]
    volume = torch.zeros(*vol_size).cpu() # TODO: confirm
    
    x = torch.Tensor(events['x'].astype('long')).cpu().long()
    y = torch.Tensor(events['y'].astype('long')).cpu().long()
    t = torch.Tensor(events['t'].astype('long')).cpu().long()
    p = torch.Tensor(events['p'].astype('long')).cpu().long()
    
    t_min = t.min()
    t_max = t.max()
    t_scaled = (t-t_min) * ((vol_size[0] // 2-1) / (t_max-t_min))
    
    ts_fl, ts_ce = calc_floor_ceil_delta(t_scaled.squeeze())
    
    inds_fl, vals_fl = create_update(x, y,
                                     ts_fl[0], ts_fl[1],
                                     p,
                                     vol_size)
    
    volume.view(-1).put_(inds_fl, vals_fl, accumulate=True)
    return volume

def video_segment_to_voxel(video, t0, delta_t, bins):
    current_time = video.current_time
    video.seek_time(t0)
    events = video.load_delta_t(delta_t)
    video.seek_time(current_time)
    return events_to_voxel(events, 5, *video.get_size())
print(video.current_time)
voxel = video_segment_to_voxel(video, 1000, delta_t=50000, bins=5)
print(video.current_time)
# https://github.com/alexzzhu/EventGAN/blob/c4230482509466741381b2d438cca1d16b497a1b/EventGAN/utils/event_utils.py#L77
events = video.load_delta_t(50000)
print(voxel.size())

0
0
torch.Size([10, 240, 304])


In [36]:
import glob
times = set()
sizes = set()
dats = glob.glob(op.join(settings['automotive_path'],"train_a","*.dat"))
for path in dats:
    video = PSEELoader(path)
    times.add(video.total_time())
    sizes.add(tuple(video.get_size()))
times = np.array(list(times))
sizes= np.array(list(sizes))
print(len(times), len(dats))
print(us_to_s(times.min()))
print(us_to_s(times.max()))
int(us_to_ms(times.min())/50)
print(len(sizes),sizes)

57 250
59.928757
59.999999
1 [[240 304]]


Assume we want int(us_to_ms(times.min())/50) voxels and bounding boxes for each voxel
dtype=[('t', '<u8'), ('x', '<f4'), ('y', '<f4'), ('w', '<f4'), ('h', '<f4'), ('class_id', 'u1'), ('class_confidence', '<f4'), ('track_id', '<u4')])

# Bounding boxes

In [7]:
video = PSEELoader(op.join(settings['automotive_path'],"train_a","17-03-30_12-53-58_1098500000_1158500000_td.dat"))
bboxes = PSEELoader(video._file.name.replace("_td.dat","_bbox.npy"))

In [ ]:
bboxes = 

In [8]:
bboxes.seek_time(0)
while not bboxes.done: 
    gt = bboxes.load_delta_t(50000)
    if gt.size:
        print(gt)

[(4099999, 162., 140., 40., 18., 0, 1., 1550)]
[(5099999, 91., 134., 43., 22., 0, 1., 1551)]
[(6099999,  13., 132., 51., 26., 0, 1., 1552)
 (6099999, 133., 137., 26., 17., 0, 1., 1553)]
[(7099999, 23., 131., 41., 23., 0, 1., 1554)
 (7099999, 60., 133., 29., 19., 0, 1., 1555)]
[(8099999,  14., 134., 35., 26., 0, 1., 1556)
 (8099999, -13., 134., 34., 28., 0, 1., 1557)]
[(10099999, 58., 134., 29., 17., 0, 1., 1558)
 (10099999, 29., 133., 39., 21., 0, 1., 1559)]
[(11099999, -6., 140., 51., 26., 0, 1., 1560)]
[(14099999, 109., 133., 30., 18., 0, 1., 1561)]
[(15099999, 65., 129., 36., 23., 0, 1., 1562)]
[(16099999, -7., 123., 53., 33., 0, 1., 1563)]
[(17099999, 289., 134., 14., 40., 1, 1., 1564)
 (17099999, -12., 144., 41., 18., 0, 1., 1565)]
[(51099999, -17., 132., 106., 88., 0, 1., 1566)]
[(52099999, -17., 128., 116., 87., 0, 1., 1567)]
[(53099999, -4., 129., 111., 70., 0, 1., 1568)]
[(54099999, 38., 135., 76., 53., 0, 1., 1569)]
[(55099999, 65., 139., 57., 39., 0, 1., 1570)]
[(56099999, 8

In [9]:
bboxes.seek_time(0)
gt = bboxes.load_delta_t(50000)
gt['x']

array([], dtype=float32)

In [10]:
# this functions can be used to read a video backwards for instance
video.seek_time(video.total_time()+1)
delta_t = 100000
for t in np.arange(video.total_time()- delta_t, -delta_t, -delta_t):
    video.seek_time(t)
    events = video.load_delta_t(delta_t)
    # they should be sorted in descending timestamp order !
    events = events[::-1]
    # do some cunning computer vision here.
    pass

In [11]:
# Generate a csv

In [34]:
#video = PSEELoader(op.join(settings['automotive_path'],"train_a","17-03-30_12-53-58_1098500000_1158500000_td.dat"))
for split in ("train_a","val_a"):
    video_filepaths = glob.glob((op.join(settings['automotive_path'],split, "*.dat")))
    lines = []
    for video_filepath in video_filepaths:
        anno_filepath = video_filepath.replace("_td.dat", "_bbox.npy")
        assert op.exists(anno_filepath)
        video = PSEELoader(video_filepath)
        anno = PSEELoader(video_filepath)
        assert video.total_time() == anno.total_time()
        total_time = video.total_time()
        line = f"{video_filepath},{anno_filepath},{total_time}\n"
        lines.append(line)
    with open(f"{split}.csv", "w") as f:
        f.writelines(lines)

'/tmp2/igor/EV/Dataset/Automotive/train_a/17-04-06_09-57-37_2806500000_2866500000_td.dat,/tmp2/igor/EV/Dataset/Automotive/train_a/17-04-06_09-57-37_2806500000_2866500000_bbox.npy,59999998\n'